# Problème des reines sur échiquier

## Genetic algorithm

In [3]:
# Dependencies
import numpy as np
import matplotlib.pyplot as plt

## 1. Solutions

- **Solutions** : Toutes les cases sont utilisées
- **Solution** optimales : Il y a 0 conflit

## 2. Implémentation

In [2]:
"""
We init the board as a 2D array of 0
0 means no queen
1 means a queen is present
2 means the cell is attacked by a queen
"""
def board(n):
    return [[0 for _ in range(n)] for _ in range(n)]

## 3. Fonction objectif

In [ ]:
"""
Transform the board to a list of queen
Index = nb line
Value = nb column
"""
def index_queen(board):
    index = []
    for i in range(len(board)):
        for j in range(len(board)):
            if board[i][j] == 1:
                index.append((i, j))
    return index

"""
Objective function: we want the optimal solution
For that we need to check the diagonal, horizontal and vertical lines
"""
def objective_function(index_list):
    # 1. No queen in corners
         
    # 2. A queen is present in the same column
    for i in range(len(index_list)):
        for j in range(i+1, len(index_list)):
            if index_list[i] == index_list[j]:
                return False
    
    # 3. A queen is present in the same row
    for i in range(len(index_list)):
        for j in range(i+1, len(index_list)):
            if abs(index_list[i] - index_list[j]) == abs(i - j):
                return False
    

## 4. Croisement

In [ ]:
# 
def cross():
    
def correct():

## 5. Mutation

## 6. Run

In [ ]:
board(4)
